In [1]:
# Table of contents:
## data preparation
## move data into s3 bucket
## create model (the object to be used train)
## train model
## deploy model

# data preparation

In [2]:
import urllib.request

urllib.request.urlretrieve('https://archive.ics.uci.edu/static/public/53/iris.zip',
                          'data.zip')

('data.zip', <http.client.HTTPMessage at 0x7fab098d1990>)

In [3]:
!mkdir data
!unzip data.zip -d data/

Archive:  data.zip
  inflating: data/Index              
  inflating: data/bezdekIris.data    
  inflating: data/iris.data          
  inflating: data/iris.names         


In [10]:

import pandas as pd

# read data
data = pd.read_csv('data/iris.data', header=None)

# convert to numerical values
data[4] = data[4].replace('Iris-setosa', 0)
data[4] = data[4].replace('Iris-virginica', 1)
data[4] = data[4].replace('Iris-versicolor', 2)

# shuffle data
data = data.sample(frac=1).reset_index(drop=True)

# change label column indices
data = data[[4, 0, 1, 2, 3]]

# split data to 'training' and 'validation' sets

# Total: 150 instances
# 80% train data (120 instances)
# 20% validation data (30 instances)

train_data = data[:120]
val_data = data[120:]

/tmp/ipykernel_1185/29921390.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[4] = data[4].replace('Iris-versicolor', 2)


# move data into s3 bucket

In [13]:
import boto3


bucket_name = 'sagemaker-build-and-deploy-model-sagemaker2'

# Save training data
train_data.to_csv('data.csv', header=False, index=False)
key = 'data/train/data'
url = 's3://{}/{}'.format(bucket_name, key)
boto3.Session().resource('s3').Bucket(bucket_name).Object(key).upload_file('data.csv')

# Save validation data
val_data.to_csv('data.csv', header=False, index=False)
key = 'data/val/data'
url = 's3://{}/{}'.format(bucket_name, key)
boto3.Session().resource('s3').Bucket(bucket_name).Object(key).upload_file('data.csv')

# create model

In [25]:
import sagemaker
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker import get_execution_role

key = 'model/xgb_model'
s3_output_location = 's3://{}/{}'.format(bucket_name, key)

xgb_model = sagemaker.estimator.Estimator(
    get_image_uri(boto3.Session().region_name, 'xgboost'),
    get_execution_role(),
    train_instance_count=1,
    train_instance_type='ml.m5.xlarge',
    train_volume_size=5,
    output_path=s3_output_location,
    sagemaker_session=sagemaker.Session()
)

xgb_model.set_hyperparameters(max_depth=5,
                             eta=0.2,
                             gamma=4,
                             min_child_weight=6,
                             silent=0,
                             objective='multi:softmax',
                             num_class=3,
                             num_round=10)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


# train model

In [26]:
train_data = 's3://{}/{}'.format(bucket_name, 'data/train')
val_data = 's3://{}/{}'.format(bucket_name, 'data/val')

train_channel = sagemaker.session.s3_input(train_data, content_type='text/csv')
val_channel = sagemaker.session.s3_input(val_data, content_type='text/csv')

data_channels = {'train': train_channel, 'validation': val_channel}

xgb_model.fit(inputs=data_channels)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker:Creating training-job with name: xgboost-2024-07-23-17-44-48-468


2024-07-23 17:44:48 Starting - Starting the training job...
2024-07-23 17:45:03 Starting - Preparing the instances for training...
2024-07-23 17:45:43 Downloading - Downloading the training image.....Arguments: train
[2024-07-23:17:46:31:INFO] Running standalone xgboost training.
[2024-07-23:17:46:31:INFO] File size need to be processed in the node: 0.0mb. Available memory size in the node: 8158.36mb
[2024-07-23:17:46:31:INFO] Determined delimiter of CSV input is ','
[17:46:31] S3DistributionType set as FullyReplicated
[17:46:31] 120x4 matrix with 480 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2024-07-23:17:46:31:INFO] Determined delimiter of CSV input is ','
[17:46:31] S3DistributionType set as FullyReplicated
[17:46:31] 30x4 matrix with 120 entries loaded from /opt/ml/input/data/validation?format=csv&label_column=0&delimiter=,
[17:46:31] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[17:46:

# deploy model

In [27]:
xgb_predictor = xgb_model.deploy(initial_instance_count=1,
                                instance_type='ml.m5.xlarge')

INFO:sagemaker:Creating model with name: xgboost-2024-07-23-17-51-03-328
INFO:sagemaker:Creating endpoint-config with name xgboost-2024-07-23-17-51-03-328
INFO:sagemaker:Creating endpoint with name xgboost-2024-07-23-17-51-03-328


-----!